# Decorators

 Decorators wrap around other functions, adding some extra functionality without
 changing the core behaviour.

 Decorators:
   1. Takes a function as an argument
   2. Returns a closure
   3. Closure typically accepts any combination of parameters (***args**, ****kwargs**)
       1. Closure runs some code in the inner function
       2. Closure calls the original function using the arguments passed
   4. Returns whatever is returned by that function call

## Python decoration syntax

``@decorator`` <br>
``def my_func():``

is equivalent to

``my_func = decorator(my_func)``



## Introspecting Decorated functions
``@wraps`` from the functools module can be used to preserve the metadata of the decorated functions.
Used in the decorator, just before the inner function.

``my_func.__name__`` can be then used to get the name of the function <br>
``help(my_func)`` can be used as before, pulling docstrings and annotations

# Decorator

In [67]:
def decorator_mwe(fn): # Decorators take in the function
    # A free variable can be defined here for storage and use.
    def inner(*args,**kwargs):
        # Typically something is done here - e.g. timing, memoization etc.
        func_result = fn(*args,**kwargs)
        return func_result # Closure returns the function result
    return inner # Decorators return the function result, with some additional functionality

In [19]:
def timed(fn): # function to be decorated is passed in
    from time import perf_counter # returns current time
    from functools import wraps # preserves metadata

    @wraps(fn) # parameterized decorator takes in function whose metadata is to be preserved
    def inner(*args,**kwargs): # take in any arguments
        start = perf_counter() # start timer
        function = fn(*args,*kwargs) # feed through and unwrap arguments
        end = perf_counter() # end timer
        time_elapsed = end-start #get time elapsed

        args_list = [str(a) for a in args]
        kwargs_list = [f'{k}:{v}' for k,v in kwargs.items()] #extract key value from dict using tuple unpack and list comp.
        print(f'Function took {time_elapsed} seconds to run.\n'
              f'for args: {args_list} and kwargs: {kwargs_list}') # Print out how long it took
        return function
    return inner

In [20]:
@timed
def addsimple(a,b):
    return a+b

addsimple(100,1234)


Function took 2.0000006770715117e-06 seconds to run.
for args: ['100', '1234'] and kwargs: []


1334

# Parameterized Decorators
These are achieved by nesting.
The outer decorator is a decorator factory that generates a decorator, passing through a parameter used during decoration.

In [49]:
def paramdec_mwe(a,b): # A decorator factory
    '''
    A minimum working example for a parameterized decorator.
    '''
    from functools import wraps
    def internal_decorator(fn): # Start of the internal decorator
        @wraps(fn)
        def inner(*args,**kwargs):
            print(f'The parameters for the decorator were {a} and {b}.')
            # The parameters a and b are passed down through all the levels as a free variable.
            function_result = fn(*args,**kwargs)
            return function_result
        return inner # End of the internal decorator.
    return internal_decorator # The decorator factory returns a decorator.


In [60]:
def time_it(cycles=100): # This is a decorator factory
    '''
    A simple parameterized decorator - timing over a number of cycles.
    Default is 100, can set as many as you want.
    '''
    # Takes in a parameter to pass through to the decorator being
    # In this case, it's the number of cycles to run the function to be timed
    from time import perf_counter # returns current time
    from functools import wraps # preserves metadata
    def inner_decorator(fn):
        @wraps(fn)
        def timer(*args,**kwargs): # take in any arguments
            start = perf_counter() # start timer
            func_result = fn(*args,*kwargs) # feed through and unwrap arguments
            end = perf_counter() # end timer
            for i in range(cycles):
                time_elapsed = end-start #get time elapsed
            args_list = [str(a) for a in args]
            kwargs_list = [f'{k}:{v}' for k,v in kwargs.items()] #extract key value from dict using tuple unpack and list comp.
            average_time = time_elapsed/cycles
            print(f'Function took {time_elapsed} seconds to run.\n'
                  f'Input args: {args_list} and kwargs: {kwargs_list}\n'
                  f'{cycles} cycles were run.\n'
                  f'The average time per cycle was {average_time}.') # Print out how long it took
            return func_result
        return timer
    return inner_decorator

## Decorator Usage


In [62]:
@time_it(1000)
def addsimple(a,b):
    return a+b

addsimple(100,1234)

Function took 3.3999995139311068e-06 seconds to run.
Input args: ['100', '1234'] and kwargs: []
1000 cycles were run.
The average time per cycle was 3.3999995139311067e-09.


1334

In [65]:
help(time_it)

Help on function time_it in module __main__:

time_it(cycles=100)
    A simple parameterized decorator - timing over a number of cycles.
    Default is 100, can set as many as you want.

